# Extracting Time and Coordinates
This python notebook uses pandas, regex and datetime to extract coordinates, get the time from emergency call to ambulance at the scene. It also gets the calls that are emergencies only.

In [ ]:
# Data was downloaded via: https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3/data
# Filtered to 2022-2023
# Name: emergency_calls.csv

In [14]:
import pandas as pd
import regex
import datetime

In [34]:
df = pd.read_csv("emergency_calls.csv")
#case_location 
# df.info()
df = df[['On Scene DtTm', 'Hospital DtTm', 'case_location', 'Call Final Disposition']]

df = df.dropna()
df.head()



/var/folders/7k/_pnpn0wd6xj6r3_6p1q9g9q00000gn/T/ipykernel_2273/442090038.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("emergency_calls.csv")


,On Scene DtTm,Hospital DtTm,case_location,Call Final Disposition
4,01/02/2023 09:12:54 PM,01/02/2023 09:41:24 PM,POINT (-122.41772042465136 37.71066123477516),Code 2 Transport
56,01/02/2023 11:04:52 AM,01/02/2023 11:37:41 AM,POINT (-122.41416369312293 37.78750449527436),Code 2 Transport
65,01/02/2023 11:40:48 PM,01/03/2023 12:02:21 AM,POINT (-122.41359351287615 37.78469892197606),Code 3 Transport
66,01/02/2023 02:25:05 PM,01/02/2023 02:50:43 PM,POINT (-122.40511819724905 37.77465347670743),Code 2 Transport
68,01/02/2023 04:09:22 PM,01/02/2023 04:46:32 PM,POINT (-122.41182302365789 37.75183124372178),Code 2 Transport


In [35]:
x_vals = []
y_vals = []
for val in df['case_location']:
    match = regex.findall(r'\((\-?\d{1,5}\.\d{1,})\s(\d{1,5}\.\d{1,})', val)
    try:
        for vals in match:
            x_vals.append(vals[0])
            y_vals.append(vals[1])
        
    except:
        pass

df['x'] = x_vals
df['y'] = y_vals
df = df.drop('case_location', axis = 1)
df.head()

,On Scene DtTm,Hospital DtTm,Call Final Disposition,x,y
4,01/02/2023 09:12:54 PM,01/02/2023 09:41:24 PM,Code 2 Transport,-122.41772042465136,37.71066123477516
56,01/02/2023 11:04:52 AM,01/02/2023 11:37:41 AM,Code 2 Transport,-122.41416369312293,37.78750449527436
65,01/02/2023 11:40:48 PM,01/03/2023 12:02:21 AM,Code 3 Transport,-122.41359351287615,37.78469892197606
66,01/02/2023 02:25:05 PM,01/02/2023 02:50:43 PM,Code 2 Transport,-122.40511819724905,37.77465347670743
68,01/02/2023 04:09:22 PM,01/02/2023 04:46:32 PM,Code 2 Transport,-122.41182302365789,37.75183124372178


In [36]:
df['Scene']= pd.to_datetime(df['On Scene DtTm'], format='%m/%d/%Y %I:%M:%S %p')
df['Hospital'] = pd.to_datetime(df['Hospital DtTm'], format='%m/%d/%Y %I:%M:%S %p')
df = df.drop('On Scene DtTm', axis = 1)
df = df.drop('Hospital DtTm', axis = 1)
df['time'] = (df['Hospital']- df['Scene'])
df.head()

,Call Final Disposition,x,y,Scene,Hospital,time
4,Code 2 Transport,-122.41772042465136,37.71066123477516,2023-01-02 21:12:54,2023-01-02 21:41:24,0 days 00:28:30
56,Code 2 Transport,-122.41416369312293,37.78750449527436,2023-01-02 11:04:52,2023-01-02 11:37:41,0 days 00:32:49
65,Code 3 Transport,-122.41359351287615,37.78469892197606,2023-01-02 23:40:48,2023-01-03 00:02:21,0 days 00:21:33
66,Code 2 Transport,-122.40511819724905,37.77465347670743,2023-01-02 14:25:05,2023-01-02 14:50:43,0 days 00:25:38
68,Code 2 Transport,-122.41182302365789,37.75183124372178,2023-01-02 16:09:22,2023-01-02 16:46:32,0 days 00:37:10


In [38]:
df['time'] = pd.to_timedelta(df['time'])
df['total time'] = df['time'].dt.total_seconds()
df = df[df['total time'] >= 0]
df.head()

,Call Final Disposition,x,y,Scene,Hospital,time,total time
4,Code 2 Transport,-122.41772042465136,37.71066123477516,2023-01-02 21:12:54,2023-01-02 21:41:24,0 days 00:28:30,1710.0
56,Code 2 Transport,-122.41416369312293,37.78750449527436,2023-01-02 11:04:52,2023-01-02 11:37:41,0 days 00:32:49,1969.0
65,Code 3 Transport,-122.41359351287615,37.78469892197606,2023-01-02 23:40:48,2023-01-03 00:02:21,0 days 00:21:33,1293.0
66,Code 2 Transport,-122.40511819724905,37.77465347670743,2023-01-02 14:25:05,2023-01-02 14:50:43,0 days 00:25:38,1538.0
68,Code 2 Transport,-122.41182302365789,37.75183124372178,2023-01-02 16:09:22,2023-01-02 16:46:32,0 days 00:37:10,2230.0
